# Dataset Mercado Inmobiliario

** Breve Intro de la creación del Dataset **

In [3]:
# Carga de librerías necesarias para el trabajo
import requests # Permite hacer llamados con el protocolo HTTP
import json # Permite trabajar con los archivos JSON
import pandas as pd # Trabajar con datos tabulares
import numpy as np # Trabajar con operaciones matemáticas y arrays

In [23]:
# Función que permite acceder a la información del URL

def get_response(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("Ops: Something Else",err)

In [24]:
# Primero indicamos la consulta que vamos a hacer
url='https://api.mercadolibre.com/sites/MLA/search?item_location=lat:-34.687056_-34.537684,lon:-58.270306_-58.528547&category=MLA1459&limit=50&offset=0#json'

offset=0 #Arrancamos con el offset 0
r = get_response(url) #Vemos los resultados de la consulta
maximum = int(str(r.json()["paging"]["total"])) #Consultamos cuál es el número máximo de resultados

print('Encontramos {} resultados para nuestra consulta'.format(maximum))

Encontramos 96552 resultados para nuestra consulta


In [26]:
# Vamos a forzarnos a buscar pocos resultados para seguir testeando y no perder tanta velocidad
# Cuando vayamos full mode, esta linea de abajo hay que comentarla o borrarla
maximum=1000


# Ahora, sabiendo esto, vamos a traer todos los resultados iterando y aumentando el offset en cada loop.
# Recuerden que solo podemos traer de a tandas de a 50 resultados
item_list = []

#Se definen los punto de inicio y final del rectángulo a barrer en búsqueda inmuebles
lat1=-34.687056
lon1=-58.270306
latF=-34.537684
lonF=-58.528547

#Se calcula la divisón del espacio a analizar
rango = min(abs(latF-lat1),abs(lonF-lon1))
division = rango/100

print("El valor del rango es: "+ str(rango))
print("El valor de la division es: "+ str(division))

El valor del rango es: 0.14937199999999962
El valor de la division es: 0.0014937199999999961


In [27]:
(r.status_code)

200

In [28]:
#Se itera de tal forma de barrer el rectángulo en pequeñas porciones
while lat1 <= latF:
    lat2=lat1+division
    
    while lon1 >= lonF:
        lon2=lon1-division
        
        while r.status_code == 200 and offset < maximum:
            url='https://api.mercadolibre.com/sites/MLA/search?item_location=lat:{}_{},lon:{}_{}&category=MLA401685&offset={}#json'.format(lat1,lat2,lon1,lon2,offset)
            r = get_response(url)
            data = r.json()
            length = len(data['results'])
            for i in range(length):
                item_id = data['results'][i]['id']
                item_list.append(item_id)
            offset += 50
                        
            #print("Porcentaje de completitud offset: {:0.2%}".format(offset/maximum),end='\r')
            #print("Porcentaje de completitud lon: {:0.2%}".format(abs(lon2/lonF)),end='\r')
            #print("Porcentaje de completitud lat: {:0.2%}".format(latF/lat2),end='\r')

        lon1 -= division
        offset=0

    lat1 += division
    offset=0
    print("Porcentaje de completitud lat: {:0.2%}".format(lat2/latF),end='\r')

len(item_list)

Error Connecting: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


AttributeError: 'NoneType' object has no attribute 'json'

In [20]:
url='https://api.mercadolibre.com/sites/MLA/search?item_location=lat:{}_{},lon:{}_{}&category=MLA401685&offset={}#json'.format(lat1,lat2,lon1,lon2,offset)
print(url)
r = get_response(url)
data = r.json()
data

https://api.mercadolibre.com/sites/MLA/search?item_location=lat:-34.687056_-34.686906627999996,lon:-58.281060784000196_-58.2812101560002&category=MLA401685&offset=350#json


{'site_id': 'MLA',
 'country_default_time_zone': 'GMT-03:00',
 'paging': {'total': 31, 'primary_results': 31, 'offset': 350, 'limit': 50},
 'results': [],
 'sort': {'id': 'relevance', 'name': 'Más relevantes'},
 'available_sorts': [{'id': 'price_asc', 'name': 'Menor precio'},
  {'id': 'price_desc', 'name': 'Mayor precio'}],
 'filters': [{'id': 'item_location',
   'name': 'Ubicación',
   'type': 'text',
   'values': [{'id': 'lat:-34.687056_-34.686906627999996,lon:-58.281060784000196_-58.2812101560002',
     'name': 'Área del mapa seleccionada'}]},
  {'id': 'category',
   'name': 'Categorías',
   'type': 'text',
   'values': [{'id': 'MLA401685',
     'name': 'Propiedades Individuales',
     'path_from_root': [{'id': 'MLA1459', 'name': 'Inmuebles'},
      {'id': 'MLA1466', 'name': 'Casas'},
      {'id': 'MLA1468', 'name': 'Venta'},
      {'id': 'MLA401685', 'name': 'Propiedades Individuales'}]}]}],
 'available_filters': [{'id': 'state',
   'name': 'Ubicación',
   'type': 'text',
   'value

In [ ]:
#Obtención de las características de cada uno de los items
final_list = []
for i in range(len(item_list)):
    item="https://api.mercadolibre.com/items/{}".format(item_list[i])
    item_add = requests.get(item)
    item_add = item_add.json()
    final_list.append(item_add)
    print("Porcentaje de completitud: {:0.2%}".format((i+1)/len(item_list)),end='\r')

In [ ]:
#Se Transforman los datos a tabla con pd.json_normalize
data=pd.json_normalize(final_list)
data.head()

In [ ]:
headers = list(data.columns)
list(data.columns.values)
#headers

In [ ]:
#Se guarda el CSV en el disco
data.to_csv('Dataset_Inmobiliario.csv', encoding='utf-8')